In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Hogyan ne járjunk be egy Pandas DataFrame-et

@dmlab2023.v2.1

### Segedfüggvények

In [2]:
def adathalmaz_generalo(oszlopszam, sorszam):
    dataset={}
    for oszlop in range(oszlopszam):
        attributum_name="att"+str(oszlop)
        dataset[attributum_name]=np.random.randint(1,100,sorszam)
    return pd.DataFrame(dataset)

In [3]:
qs=adathalmaz_generalo(2,4)

In [4]:
qs.T

,0,1,2,3
att0,66,4,69,39
att1,29,85,39,21


In [5]:
qs

,att0,att1
0,66,29
1,4,85
2,69,39
3,39,21


### Bejarasi strategiakat megvalosító függvények

In [6]:
def egyszeru_forciklus(df,oszlopnev="att0"):
    df['new']=0
    for i in range(len(df)):
        adat = df.loc[i,oszlopnev]
        if adat%2==0:
            df.loc[i,'new']='paros'
        else:
            df.loc[i,'new']='paratlan'
    return df

In [7]:
def okos_forciklus(df,oszlopnev="att0"):
    result=[]
    for i,r in df.iterrows():
        adat = r[oszlopnev]
        if adat%2==0:
            result.append('paros')
        else:
            result.append('paratlan')
    df['new']=result
    return df
    

In [8]:
def pandas_apply(df,oszlopnev="att0"):
    def fv(adat):
        if adat%2==0:
            return 'paros'
        else:
            return 'paratlan'        
    df['new']=df[oszlopnev].apply(fv)
    return df

# Futási idők különböző módszerek esetén

In [9]:
#100 sor eseten
df=adathalmaz_generalo(2,100)

In [10]:
%timeit egyszeru_forciklus(df)

41.8 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit okos_forciklus(df)

9.78 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit pandas_apply(df)

557 µs ± 674 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
#1000 sor eseten
df=adathalmaz_generalo(2,1000)

In [14]:
%timeit egyszeru_forciklus(df)

435 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit okos_forciklus(df)

94.8 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit pandas_apply(df)

827 µs ± 6.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
#10000 sor eseten
df=adathalmaz_generalo(2,10000)

In [18]:
%timeit egyszeru_forciklus(df)

6.15 s ± 7.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit okos_forciklus(df)

943 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit pandas_apply(df)

3.33 ms ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
# 1 millio sor eseten
df=adathalmaz_generalo(2,1000000)

Ez -n1 -r1 paraméterekkel itt azt érjük el, hogy a futtatás egyetlen egyszer fusson csak le.

Ugyanakkor az egyszerű és okos megoldást itt már nem is érdemes elindítani, mert nem fog lefutni több perc alatt sem.

In [22]:
# %timeit -n1 -r1 egyszeru_forciklus(df)

In [23]:
# %timeit -n1 -r1 okos_forciklus(df)

In [24]:
%timeit -n1 -r1 pandas_apply(df)

289 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Konklúzió

Ha mód van rá kerüljük el, hogy a dataframe elemeit nekünk kelljen egy ciklussal bejárni, hiszen ettől az műveletek extrém hosszú futási idejűek lesznek. Egy megfelelő pandas apply függvénynél kb. 600-szor leszel lassabb, ha a szintén beépített iterrows függvényt használod a tömb bejárására, de úgy 2500-szor leszel lassabb, ha simán bejárod egy for (vagy while) ciklussal a tömbödet. 


# A futási eredmények egy ábrára összerakva

In [27]:
# Ez nagyon sokáig futhat, így ezt alapból nem futtatjuk le
if False:
    x_tengely=[]
    y_tengely=[]
    #maxtime=1
    maxtime=5
    for fv in [pandas_apply,okos_forciklus,egyszeru_forciklus]:
        fvnev=(str(fv).split(" "))[1]
        print(fvnev)
        x_tengely=[]
        y_tengely=[]
        for i in range(10):
            size = 10**(i+1)
            df=adathalmaz_generalo(2,size)
            if i<4:
                result = %timeit -o fv(df) 
            else:
                result = %timeit -n1 -r1 -o fv(df) 
            sec = result.best
            #print(fvnev,size,sec)
            x_tengely.append(size)
            y_tengely.append(sec)
            if maxtime<sec:
                break
        plt.plot(x_tengely,y_tengely,marker='.',label=fvnev)
    plt.yscale("log")
    plt.ylabel("Futási idő (sec)")
    plt.xscale("log")
    plt.xlabel("Dataframe hossza (sor)")
    plt.legend()
    plt.savefig("data/dataframe_bejaras"+str(maxtime)+".png")


### Képfileból betöltve, mert sokáig futhat a fenti ábra generáló
<img src="data/dataframe_bejaras5.png">